In [1]:
import torch
import numpy as np
from PIL import Image
from IPython.display import display

In [3]:
from models import Generator
from config import config, update_config

In [5]:
class Args:
    cfg = 'experiments/itri.yml'
config = update_config(config, Args())

In [ ]:
latent_size = 512

g_ema = Generator(512, 0)